## KR260 HLS Configurable Fixed Gain with AXI Streaming Interface
This tutorial demonstrates how to accelerate a Python function on the KR260 using PYNQ, and how to interface with a Vitis HLS modules which has a AXI Streaming interface with an AXI Lite interface for Status/Control

### Plot function for use in this notebook
The first code block below defines a function that we will use for plotting data throughout this notebook. Note that the function has a `n_samples` argument so that we can limit the number of samples to plot. Plotting more than a few thousand samples can be very slow and consume a lot of RAM.

### Requirements
Install the following:

```bash
pip install ipympl
pip install ipywidgets
pip install jupyter_bokeh
```

# Start of Demo Application

#### Bokeh Plot Function

In [1]:
from bokeh.plotting import figure, output_file, show
from bokeh.io import output_notebook, show
import numpy as np
import math


def plot_to_notebook(time_sec,in_signal,n_samples,out_signal=None):
    output_notebook()
    p = figure(title = "Tone Plot", x_axis_label = 'Time (usec)', y_axis_label = 'Mag (+/-16bit)')
    p.line(time_sec[:n_samples]*1e6,in_signal[:n_samples],legend_label="Input Signal",line_color="blue",line_width=2)
    if out_signal is not None:
        p.line(time_sec[:n_samples]*1e6,out_signal[:n_samples],legend_label="Gain Output",line_color="red",line_width=2)
    show(p)

##### MatPlotLib Plot Function (Unused)

```python
%matplotlib inline
import matplotlib.pyplot as plt

def plot_to_notebook(time_sec,in_signal,n_samples,out_signal=None):
    plt.figure()
    plt.subplot(1, 1, 1)
    plt.xlabel('Time (usec)')
    plt.grid()
    plt.plot(time_sec[:n_samples]*1e6,in_signal[:n_samples],'y-',label='Input signal')
    if out_signal is not None:
        plt.plot(time_sec[:n_samples]*1e6,out_signal[:n_samples],'g-',linewidth=2,label='Gain output')
    plt.legend()
```

### Generate the input signal

In [2]:
import numpy as np
# Amp
AMP = 4096
# Total time
T = 0.02
# Sampling frequency
fs = 100e6
# Number of samples
n = int(T * fs)
# Time vector in seconds
t = np.linspace(0, T, n, endpoint=False)
# Samples of the signal
samples = 4096*np.sin(0.2e6*2*np.pi*t) # + 1500*np.cos(46e6*2*np.pi*t) + 2000*np.sin(12e6*2*np.pi*t)
# Convert samples to 32-bit integers
samples = samples.astype(np.int32)
print('Number of samples: ',len(samples))

# Plot signal to the notebook
plot_to_notebook(t,samples,2000)

Number of samples:  2000000


Loading BokehJS ...

### Hardware HLS Gain Block Implementation
##### Load Overlay

In [ ]:
from pynq import Overlay
from pynq import allocate
from pynq import MMIO
import pynq.lib.dma
import time

#import Python library from repo:
import sys
sys.path.append('/home/root/jupyter_notebooks/puch/')
import fpga.py.puch
import fpga.lib.timestamp.sw.timestamp_regmap
import fpga.lib.led_reg.sw.led_regmap



# Load the overlay
overlay = Overlay('../../overlays/KR260_HLS_fixed_gain_stream/kr260_hls_fixed_gain_stream.bit')

# Assign blocks to short vars
dma          = overlay.axi_dma_0
gain_module  = overlay.axi_fixed_gain_stream_0
led_module   = led_regmap.RegMap(puch.PynqInterface(overlay.led_reg_0))
timestamp    = timestamp_regmap.RegMap(puch.PynqInterface(overlay.Timestamp_0))

##### Print out the register map for DMA and HLS Gain Blocks

In [ ]:
overlay.ip_dict

In [ ]:
dma.register_map

In [ ]:
gain_module.register_map

##### Display Time Stamp Register
The time stamp is burned into the FPGA during the build process

In [4]:
print("FPGA Build Timestamp:  " + puch.get_timestamp_str(timestamp))

FPGA Build Timestamp:  2023/5/9 2:51:57


##### Configure the GAIN register Control Register
Want to enable auto restart, so the FPGA block will always be read to receive samples

In [5]:
gain_module.register_map.CTRL.AUTO_RESTART=1
gain_module.register_map.CTRL.AP_START=1
gain_module.register_map.CTRL

Register(AP_START=1, AP_DONE=0, AP_IDLE=0, AP_READY=0, RESERVED_1=0, AUTO_RESTART=1, RESERVED_2=0, INTERRUPT=0, RESERVED_3=0)

##### Toggle USER_LED[1:0] on/off

In [ ]:
for i in range(10):
    led_module.user_leds = 0x1
    time.sleep(0.25)
    led_module.user_leds = 0x2
    time.sleep(0.25)
led_module.user_leds = 0x0

##### Configure the GAIN register Value Register

In [8]:
gain_module.write(gain_module.register_map.gain.address,4)
print("Gain Register : " + str(gain_module.read(gain_module.register_map.gain.address)))

Gain Register : 4


### DMA Transfer the Sinewave buffer from ARM Processor to the FPGA HLS Gain Block, and plot

In [9]:
# Read Gain Register
print("Gain Register                : " + str(gain_module.read(gain_module.register_map.gain.address)))

print("DMA Transfer Size            : " + str(len(samples)))

# Allocate buffers for the input and output signals
in_buffer = allocate(shape=(len(samples),), dtype=np.int32)
out_buffer = allocate(shape=(len(samples),), dtype=np.int32)

# Copy the samples to the in_buffer
np.copyto(in_buffer,samples)

# Trigger the DMA transfer and wait for the result
import time
start_time = time.time()
dma.sendchannel.transfer(in_buffer)
#print('Submit sending the transfer into in_buffer')
dma.recvchannel.transfer(out_buffer)
#print('Submit receiving the transfer from out_buffer')
dma.sendchannel.wait()
#print('Done Sending the transfer to in_buffer')
dma.recvchannel.wait()
#print('Done receiving the transfer from out_buffer')
stop_time = time.time()
hw_exec_time = stop_time-start_time

print('DMA Transfer Execution Time  :',hw_exec_time,' sec')

# Plot to the notebook
plot_to_notebook(t,samples,2000,out_signal=out_buffer)

# Free the buffers
in_buffer.close()
out_buffer.close()

Gain Register                : 4
DMA Transfer Size            : 2000000
DMA Transfer Execution Time  : 0.09255266189575195  sec


Loading BokehJS ...

In [ ]:
del in_buffer, out_buffer